## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [1]:
! uv pip install agentics-py
! uv pip install 'crewai[tools]'

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 14ms
Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 10ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [2]:
from ddgs import DDGS
from crewai.tools import tool
## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("web_earch")
def web_search(query: str) -> str:
    """Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))    


## Define your agent, task and crew

Note that the agent is provided with memory = True This provides the agent with conversation history. By activating memory=True in the crew, we make sure that the agents persists across multiple crew session calls, with different input states. 

In [ ]:
from crewai import Agent, Task, Crew, Process
from agentics import AG

# Create a conversational agent with a friendly role/goal
chat_agent = Agent(
    role="Helpful Assistant",
    goal="Have a natural multi-turn conversation",
    backstory="You are a friendly assistant that remembers context and asks for clarification when needed.",
    llm=AG.get_llm_provider("gemini"),
    memory=True  # enable memory for conversational context. 
)

# Define a simple Task that represents a single AI response
task = Task(
    description="Respond appropriately to user's message, maintaining context. {input}",
    expected_output="A conversational reply",
    agent=chat_agent,
    tools=[web_search],
)

crew = Crew(
    agents=[chat_agent],
    tasks=[task],
    memory=False,  # ensure conversation state persists across turns
)

## Run the crew in a conversation loop
Since memory has been set to True, the crew will get access to the previous round of conversations, exibiting a conversational behaviour

In [4]:
conversation = ""
while user_input := input("USER: ").strip():
    print("USER:", user_input)
    result = crew.kickoff(inputs={"input": conversation + user_input})
    print("AI:", result)
    conversation += f"User>{user_input}\nAI>{result}\n"

USER: Who is alfio gliozzo?
AI: Alfio Gliozzo is an Italian researcher and professor known for his work in the fields of computer science and artificial intelligence. He has contributed to areas such as machine learning, data mining, and knowledge discovery, publishing numerous papers in international journals and conferences. Gliozzo is also involved in academic teaching and has held positions at several Italian universities, where he mentors graduate students and collaborates on interdisciplinary research projects. His expertise is recognized in both the academic community and industry, where he often consults on AI-driven solutions.
USER: Who is alfio gliozzo?
AI: Alfio Gliozzo is an Italian computer scientist and professor at the University of Bologna. He specializes in artificial intelligence, focusing on areas such as knowledge representation, reasoning, and machine learning. Over his career, he has authored many research papers, led various AI research projects, and contributed 

InternalServerError: litellm.InternalServerError: watsonxException - {"errors":[{"code":"downstream_request_failed","message":"Downstream vllm request failed: ","more_info":"https://cloud.ibm.com/apidocs/watsonx-ai#text-chat"}],"trace":"cceee6fe950144045e80792d81202ddc","status_code":500}